Text Feature Classification
===

Text feature classification of reverts. Messing around with large linear models of text features.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

In [2]:
import os
from tqdm import tqdm
import bz2
import sqlite3
import difflib
import gzip
import json
import base64
import pickle
import re
import hashlib
from datetime import datetime
from datetime import timezone
import nltk
import scipy.stats
import para
from itertools import groupby
from collections import Counter

In [24]:
import deltas
from deltas.tokenizers import wikitext_split

In [4]:
import sklearn
import sklearn.ensemble
import sklearn.metrics
import sklearn.calibration

In [5]:
git_root_dir = !git rev-parse --show-toplevel
git_root_dir = git_root_dir[0]
git_root_dir

'/export/scratch2/levon003/repos/wiki-ores-feedback'

In [6]:
raw_data_dir = "/export/scratch2/wiki_data"
derived_data_dir = os.path.join(git_root_dir, "data", "derived")
raw_data_dir, derived_data_dir

('/export/scratch2/wiki_data',
 '/export/scratch2/levon003/repos/wiki-ores-feedback/data/derived')

In [7]:
stub_history_dir = os.path.join(derived_data_dir, 'stub-history-all-revisions')
stub_history_dir

'/export/scratch2/levon003/repos/wiki-ores-feedback/data/derived/stub-history-all-revisions'

In [8]:
revision_sample_dir = os.path.join(derived_data_dir, 'revision_sample')
working_dir = os.path.join(derived_data_dir, 'audit')
working_dir

'/export/scratch2/levon003/repos/wiki-ores-feedback/data/derived/audit'

### Read sample data

In [9]:
# read in the sample dataframe
s = datetime.now()
revision_sample_dir = os.path.join(derived_data_dir, 'revision_sample')
sample3_filepath = os.path.join(revision_sample_dir, 'sample3_all.pkl')
rev_df = pd.read_pickle(sample3_filepath)
print(f"Sample 3 data loaded in {datetime.now() - s}.")
len(rev_df)

Sample 3 data loaded in 0:00:28.522013.


33964442

In [10]:
rev_df.head()

,page_id,rev_id,rev_timestamp,is_revert_target,is_reverted,is_reverting,is_sample_eligible,prev_rev_id,next_rev_id,prev_rev_timestamp,next_rev_timestamp,reverted_rev_ids,reverting_rev_id,reverting_rev_timestamp
1,12,818613649,1515102279,0,0,0,True,818611292,818624114,1515101356,1515106953,[],-1,-1
2,12,818624114,1515106953,1,0,0,True,818613649,820024812,1515102279,1515798752,[],-1,-1
3,12,820024812,1515798752,0,1,0,True,818624114,820025687,1515106953,1515799060,[],820025687,1515799060
4,12,820025687,1515799060,0,0,1,True,820024812,820703495,1515798752,1516095884,[820024812],-1,-1
5,12,820703495,1516095884,0,0,0,True,820025687,821673418,1515799060,1516597634,[],-1,-1


### Load texts into memory

In [11]:
audit_dir = os.path.join(derived_data_dir, 'audit')
text_db_filepath = os.path.join(audit_dir, 'text_2020-07-23T13:08:38Z.sqlite')

In [12]:
def get_db(db_filename):
    db = sqlite3.connect(
            db_filename,
            detect_types=sqlite3.PARSE_DECLTYPES
        )
    db.row_factory = sqlite3.Row
    return db

In [13]:
#text_dict_list = []
rev_id_content_dict = {}
rev_id_comment_dict = {}
try:
    db = get_db(text_db_filepath)
    cursor = db.execute("SELECT rev_id, content, comment FROM revisionText")
    for result in tqdm(cursor, total=1106018):
        rev_id = result['rev_id']
        rev_id_content_dict[rev_id] = result['content']
        rev_id_comment_dict[rev_id] = result['comment']
        #comment = result['comment']
        #content = result['content']
        #text_dict_list.append({
        #    'rev_id': rev_id,
        #    'content': content,
        #    'comment': comment
        #})
finally:
    db.close()
len(rev_id_content_dict)

100%|██████████| 1106018/1106018 [02:53<00:00, 6362.74it/s]


1106018

In [22]:
#text_df = pd.DataFrame(text_dict_list)
#print(len(text_df))
#text_df.head()

### Add text availability to sample3 revision data

Either join in a dataframe with the text data or just record which entries have text available.

In [15]:
#df = pd.merge(rev_df, text_df, how='left', on='rev_id')
df = rev_df

In [17]:
#df['has_text'] = ~df.content.isna()
df['has_text'] = df.rev_id.map(lambda rev_id: rev_id in rev_id_content_dict)
np.sum(df.has_text), np.sum(df.has_text) / len(df)

(1106018, 0.032563997371133024)

In [18]:
rev_ids_with_text = set(df[df.has_text].rev_id)
df['prev_rev_has_text'] = df.prev_rev_id.map(lambda rev_id: rev_id in rev_ids_with_text)

In [19]:
np.sum(df.prev_rev_has_text), np.sum(df.prev_rev_has_text) / len(df)

(996951, 0.029352786069619517)

In [20]:
np.sum((df.prev_rev_has_text)&(df.has_text))

689050

### Mess around with creating some features

In [21]:
sdf = df[(df.prev_rev_has_text)&(df.has_text)]
len(sdf)

689050

In [23]:
prev_rev_id = sdf.iloc[0].prev_rev_id
curr_rev_id = sdf.iloc[0].rev_id
prev_content = rev_id_content_dict[prev_rev_id]
curr_content = rev_id_content_dict[curr_rev_id]
len(prev_content), len(curr_content)

(154493, 154373)

In [26]:
prev_tokens = wikitext_split.tokenize(prev_content)
curr_tokens = wikitext_split.tokenize(curr_content)

In [39]:
diff = Counter(curr_tokens)
curr_counter = Counter(curr_tokens)
prev_counter = Counter(prev_tokens)
diff.subtract(prev_counter)
len(diff), len(curr_counter), len(prev_counter)

(5439, 5439, 5439)

In [52]:
for token, count in diff.items():
    if count != 0:
        print(f"{repr(token):>40}\t{count}")

           Token(' ', type='whitespace')	-18
                  Token('"', type='etc')	-2
           Token('(', type='paren_open')	-1
          Token(')', type='paren_close')	-1
            Token('2018', type='number')	-1
                Token('or', type='word')	-1
                Token('is', type='word')	-1
              Token('with', type='word')	-1
                Token(',', type='comma')	-1
         Token('treatment', type='word')	-1
              Token('some', type='word')	-1
          Token('children', type='word')	-1
               Token('ASD', type='word')	-1
            Token('review', type='word')	-1
                Token('to', type='word')	-1
              Token('that', type='word')	-1
                Token('an', type='word')	-1
        Token('considered', type='word')	-1
              Token('very', type='word')	-1
               Token('low', type='word')	-1
         Token('effective', type='word')	-1
         Token('According', type='word')	-1
              Token('EIBI', typ

In [55]:
rev_id_tokens_dict = {}
c = 0
MAX_TEXTS = 10000
for row in tqdm(sdf.itertuples(), total=len(sdf)):
    prev_rev_id = row.prev_rev_id
    curr_rev_id = row.rev_id
    if prev_rev_id not in rev_id_tokens_dict:
        prev_content = rev_id_content_dict[prev_rev_id]
        rev_id_tokens_dict[prev_rev_id] = wikitext_split.tokenize(prev_content)
        c += 1
    if curr_rev_id not in rev_id_tokens_dict:
        curr_content = rev_id_content_dict[curr_rev_id]
        rev_id_tokens_dict[curr_rev_id] = wikitext_split.tokenize(curr_content)
        c += 1
    if c >= MAX_TEXTS:
        break
len(rev_id_tokens_dict)

  1%|          | 8262/689050 [08:29<11:39:33, 16.22it/s] 


10001

In [56]:
word_counts = Counter()
for rev_id, tokens in tqdm(rev_id_tokens_dict.items(), total=len(rev_id_tokens_dict)):
    word_counts.update(tokens)
len(word_counts)

100%|██████████| 10001/10001 [00:45<00:00, 219.15it/s]


608805

In [57]:
word_counts.most_common(20)

[(Token(' ', type='whitespace'), 96745876),
 (Token('|', type='bar'), 12301610),
 (Token('=', type='equals'), 7189727),
 (Token(',', type='comma'), 6356344),
 (Token('.', type='period'), 5404204),
 (Token('the', type='word'), 4853522),
 (Token(']]', type='dbrack_close'), 4727430),
 (Token('[[', type='dbrack_open'), 4727351),
 (Token('of', type='word'), 3405190),
 (Token('\n', type='whitespace'), 3273284),
 (Token('-', type='etc'), 2802844),
 (Token('and', type='word'), 2526688),
 (Token('}}', type='dcurly_close'), 1947002),
 (Token('{{', type='dcurly_open'), 1945417),
 (Token('in', type='word'), 1931983),
 (Token("''", type='italic'), 1592096),
 (Token('to', type='word'), 1584792),
 (Token('"', type='etc'), 1481191),
 (Token(')', type='paren_close'), 1379058),
 (Token('(', type='paren_open'), 1377848)]

In [69]:
len([1 for v in word_counts.values() if v >= 100])

50895

In [71]:
labeled_rev_ids = set()
for row in tqdm(sdf.itertuples(), total=len(sdf)):
    prev_rev_id = row.prev_rev_id
    curr_rev_id = row.rev_id
    if prev_rev_id in rev_id_tokens_dict and curr_rev_id in rev_id_tokens_dict:
        labeled_rev_ids.add(curr_rev_id)
len(labeled_rev_ids)

100%|██████████| 689050/689050 [00:02<00:00, 263890.11it/s]


8263

In [73]:
prev_rev_id_dict = {row.rev_id: row.prev_rev_id for row in sdf.itertuples()}

In [72]:
n_features = len([1 for v in word_counts.values() if v >= 100])
n_features

50895

In [74]:
token_index_dict = {tup[0]: i for i, tup in enumerate(word_counts.most_common(n_features))}
len(token_index_dict)

50895

In [77]:
X = np.zeros((len(labeled_rev_ids),n_features))
for row, curr_rev_id in tqdm(enumerate(labeled_rev_ids), total=len(labeled_rev_ids)):
    prev_rev_id = prev_rev_id_dict[rev_id]
    prev_tokens = rev_id_tokens_dict[prev_rev_id]
    curr_tokens = rev_id_tokens_dict[curr_rev_id]
    diff = Counter(curr_tokens)
    prev_counter = Counter(prev_tokens)
    diff.subtract(prev_counter)
    
    for token, count in diff.items():
        if count != 0 and word_counts[token] >= 100:
            X[row,token_index_dict[token]] = count
    X[row,:] /= max(len(curr_tokens), len(prev_tokens))

100%|██████████| 8263/8263 [01:28<00:00, 93.58it/s] 


In [78]:
X.shape

(8263, 50895)

In [79]:
is_reverted_dict = {row.rev_id: row.is_reverted == 1 for row in sdf.itertuples()}
y = np.array([is_reverted_dict[rev_id] for rev_id in labeled_rev_ids])
y.shape

(8263,)

In [100]:
np.sum(y), np.sum(y) / len(y)

(1599, 0.19351325184557666)

In [82]:
np.sum(X == 0)

382788226

In [83]:
# 91% of entries are 0
382788226 / (8263 * 50895)

0.9102185867525333

In [84]:
clf = sklearn.linear_model.LogisticRegression(
    penalty='l2',
    C=1.0,
    solver='lbfgs'
)

In [85]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.20, random_state=500)

In [86]:
s = datetime.now()
print(clf)

    
# train the model
md = clf.fit(X_train, y_train)

print(f"{datetime.now() - s}")

LogisticRegression()
0:00:05.927926


In [87]:
# predict with the model
y_pred_test = md.predict(X_test)
y_pred_test_proba = md.predict_proba(X_test)[:,1]

In [88]:
np.sum(y_test == y_pred_test) / len(y_test)

0.8039927404718693

In [89]:
roc_auc = sklearn.metrics.roc_auc_score(y_test, y_pred_test_proba)
roc_auc

0.5690635769956061

In [95]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [92]:
# construct the vocabulary on all of the text documents
# this should only include TRAINING documents, not TESTING documents
s = datetime.now()

def dummy(doc):
    return doc

count_vectorizer = CountVectorizer(
        tokenizer=dummy,
        preprocessor=dummy,
        max_features=40000
    )

count_vectorizer.fit(rev_id_tokens_dict.values())
print(f"{datetime.now() - s}")

# this is the size of the vocabulary
len(count_vectorizer.vocabulary_)

/export/scratch2/levon003/bin/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


40000

In [96]:
X_docs = []
for curr_rev_id in tqdm(labeled_rev_ids):
    X_docs.append(rev_id_tokens_dict[curr_rev_id])
X = count_vectorizer.transform(X_docs)

100%|██████████| 8263/8263 [00:00<00:00, 474916.88it/s]


In [97]:
s = datetime.now()
tfidf = TfidfTransformer()
tfidf.fit(X)
print(f"{datetime.now() - s}")

0:00:00.231109


In [99]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.20, random_state=500)
s = datetime.now()

clf = sklearn.linear_model.LogisticRegression(
    penalty='l2',
    C=1.0,
    solver='lbfgs',
    max_iter=1000
)
    
# train the model
md = clf.fit(X_train, y_train)

print(f"{datetime.now() - s}")

0:02:03.295813


/export/scratch2/levon003/bin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [101]:
y_pred_test = md.predict(X_test)
y_pred_test_proba = md.predict_proba(X_test)[:,1]

pct_predicted_reverted = np.sum(y_pred_test) / len(y_pred_test)
test_acc = np.sum(y_test == y_pred_test) / len(y_test)
roc_auc = sklearn.metrics.roc_auc_score(y_test, y_pred_test_proba)
pct_predicted_reverted, test_acc, roc_auc

(0.09618874773139746, 0.7900786448880823, 0.7026598017631376)

In [103]:
X[0,:]

<1x40000 sparse matrix of type '<class 'numpy.int64'>'
	with 2918 stored elements in Compressed Sparse Row format>